# Numeric Table Detection


The objective is to demonstrate detection of tables in raw texxt to use targeted table extraction and lower the cost of using Textract.
Exmple for 1M pages per month containing 19% of pages with at leaqst 1 table. 

https://calculator.aws/#/estimate?id=fb01232c5a8b8304d1c1169b2911af087467ea82



# Notebook Setup

In this step, we will import some necessary libraries that will be used throughout this notebook. 

In [ ]:
!python -m pip install -q amazon-textract-response-parser==0.1.44
!python -m pip install -q amazon-textract-caller==0.0.28
!python -m pip install -q amazon-textract-prettyprinter==0.0.16
!python -m pip install -q amazon-textract-textractor==1.1.1
!python -m pip install -q pypdf==3.1.0
!python -m pip install -q pandas==1.3.5

In [ ]:
import boto3
import botocore
from IPython.display import Image, display, JSON, HTML
from textractcaller.t_call import call_textract, Textract_Features, call_textract_expense
from textractprettyprinter.t_pretty_print import convert_table_to_list
from trp import Document
import os


# variables

s3=boto3.client('s3')
# because the reference document is stored in us-east-2, we run Textract in us-east-2 as well
textract = boto3.client('textract', region_name='us-east-2')


In [ ]:
file="./DemoTable.pdf"
s3url="s3://amazon-textract-public-content/code-samples/DemoTable.pdf"
!aws s3 cp s3://amazon-textract-public-content/code-samples/DemoTable.pdf .
!aws s3 cp s3://amazon-textract-public-content/code-samples/DemoTableDocument-ExtractedTablePages.pdf .

In [ ]:
from IPython.display import IFrame
IFrame(file, width=1500, height=600)

---
Extract unstructured data with Amazon Textract 

Amazon Textract is an ML powered OCR service that is capable of detecting and extracting text from documents. Text data in the form of WORDS and LINES can be extracted from documents using Amazon Textract `DetectDocumentText` API. Let's extract the words and lines from the demonstration document.

In [ ]:
# Call Amazon Textract
response = call_textract(input_document=s3url, boto3_textract_client=textract)

text=""
# Print detected text
for item in response["Blocks"]:
    if item["BlockType"] == "LINE":
        text=text+"\n"+item["Text"]
print (text)

---
# Detect Tables in raw text

In this example using a threshold of 4 characters out of the 15 most used characters being digits works well to identify if the page is mostly constituted of numeric tables.



In [ ]:
from collections import Counter
import io

def containsatable(textcontent):
    counter = Counter(textcontent)
    mostcom = counter.most_common(15)
    countdigit=0
    for i in mostcom :
        if (i[0].isdigit()):
            countdigit=countdigit+1
    return(countdigit>=4)

# Alternatively, we can identify if 50% of the lines satisfy the criteria (to identify tables embedded in text)

def containsatableperline(textcontent):
    countlines=0
    tablelines=0
    for line in io.StringIO(textcontent):
        countlines=countlines+1
        counter = Counter(line)
        mostcom = counter.most_common(15)
        countdigit=0
        for i in mostcom :
            if (i[0].isdigit()):
                countdigit=countdigit+1
        if (countdigit>=4):
            tablelines=tablelines+1
    
    return(tablelines>(countlines/2))


In [ ]:
from trp import Document
import math
doc = Document(response)
#print(doc.pages[0])
# Iterate over elements in the document
pagenum=0
pageswithtables=[]
pageswithtables2=[]
for page in doc.pages:
    pagenum=pagenum+1
    text=''
    for line in page.lines:
        text=text+line.text
    if (containsatable(text)):
        pageswithtables.append(pagenum)
    if (containsatableperline(text)):
        pageswithtables2.append(pagenum)

print("List of pages with Tables:")        
print (pageswithtables)
print (pageswithtables2)
print("Percentage of pages with tables:")    
print (str(len(pageswithtables))+"/"+str(pagenum))
print (str(math.trunc(len(pageswithtables)/pagenum*100))+"%")
    


# Call Analyse Documents only on the pages with tables
First, we build a smaller PDF document containing only the pages identified as containing tables and then we run the AnalyzeDocument call on the new document.

In [ ]:
from pypdf import PdfReader, PdfWriter

pdf_file_path = file
file_base_name = pdf_file_path.replace('.pdf', '')

pdf = PdfReader(pdf_file_path)

pages = pageswithtables
pdfWriter = PdfWriter()

for page_num in pages:
    pdfWriter.add_page(pdf.pages[page_num-1])

pdf_out = open("temp.pdf", 'wb')
pdfWriter.write(pdf_out)
pdf_out.close()


In [ ]:
from IPython.display import IFrame
IFrame('temp.pdf', width=1500, height=600)

In [ ]:
# When you are using your own document, you need to copy the result to S3 and process that.
# In our sample case, we already uploaded the output to S3, so the next line is commented out
#!aws s3 cp temp.pdf s3://{data_bucket}/temp.pdf --only-show-errors
s3url="s3://amazon-textract-public-content/code-samples/DemoTableDocument-ExtractedTablePages.pdf"

In [ ]:
import pandas as pd
resp = call_textract(input_document=s3url, features=[Textract_Features.TABLES])
tdoc = Document(resp)
dfs = list()

for page in tdoc.pages:
    for table in page.tables:
        tab_list = convert_table_to_list(trp_table=table)
        print(tab_list)
        dfs.append(pd.DataFrame(tab_list))

df1 = dfs[0]
df2 = dfs[1]

In [ ]:
df1

In [ ]:
df2